In [1]:
# Add this in a Google Colab cell to install the correct version of Pytorch Geometric.
# https://gist.github.com/ameya98
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install torch-scatter==latest+{CUDA}     -f https://pytorch-geometric.com/whl/torch-{TORCH}.html
!pip install torch-sparse==latest+{CUDA}      -f https://pytorch-geometric.com/whl/torch-{TORCH}.html
!pip install torch-cluster==latest+{CUDA}     -f https://pytorch-geometric.com/whl/torch-{TORCH}.html
!pip install torch-spline-conv==latest+{CUDA} -f https://pytorch-geometric.com/whl/torch-{TORCH}.html
!pip install torch-geometric 
!pip install mendeleev
# !pip install rdkit

Looking in links: https://pytorch-geometric.com/whl/torch-1.7.0.html
     |████████████████████████████████| 11.9MB 11.9MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.7.0.html
     |████████████████████████████████| 24.3MB 1.2MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.7.0.html
     |████████████████████████████████| 21.5MB 1.4MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.7.0.html
     |████████████████████████████████| 6.4MB 7.3MB/s 
     |████████████████████████████████| 194kB 8.6MB/s 
     |████████████████████████████████| 235kB 13.0MB/s 
     |████████████████████████████████| 2.2MB 13.8MB/s 
     |████████████████████████████████| 51kB 6.0MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-1.6.3-cp36-none-any.whl size=322720 sha256=13108ce8b16b71e7acd26c2e6b2a900905a922b0013f089e4119cda16746ed27
  Stored in directory: /root/.cache/pip/wheels/6d/47/1e/0af8ce3e21783c3e584c22502011a3367c091694eebc50a971
Su

In [2]:
import sys
import os
import requests
import subprocess
import shutil
from logging import getLogger, StreamHandler, INFO


logger = getLogger(__name__)
logger.addHandler(StreamHandler())
logger.setLevel(INFO)


def install(
        chunk_size=4096,
        file_name="Miniconda3-latest-Linux-x86_64.sh",
        url_base="https://repo.continuum.io/miniconda/",
        conda_path=os.path.expanduser(os.path.join("~", "miniconda")),
        rdkit_version=None,
        add_python_path=True,
        force=False):
    """install rdkit from miniconda
    ```
    import rdkit_installer
    rdkit_installer.install()
    ```
    """

    python_path = os.path.join(
        conda_path,
        "lib",
        "python{0}.{1}".format(*sys.version_info),
        "site-packages",
    )

    if add_python_path and python_path not in sys.path:
        logger.info("add {} to PYTHONPATH".format(python_path))
        sys.path.append(python_path)

    if os.path.isdir(os.path.join(python_path, "rdkit")):
        logger.info("rdkit is already installed")
        if not force:
            return

        logger.info("force re-install")

    url = url_base + file_name
    python_version = "{0}.{1}.{2}".format(*sys.version_info)

    logger.info("python version: {}".format(python_version))

    if os.path.isdir(conda_path):
        logger.warning("remove current miniconda")
        shutil.rmtree(conda_path)
    elif os.path.isfile(conda_path):
        logger.warning("remove {}".format(conda_path))
        os.remove(conda_path)

    logger.info('fetching installer from {}'.format(url))
    res = requests.get(url, stream=True)
    res.raise_for_status()
    with open(file_name, 'wb') as f:
        for chunk in res.iter_content(chunk_size):
            f.write(chunk)
    logger.info('done')

    logger.info('installing miniconda to {}'.format(conda_path))
    subprocess.check_call(["bash", file_name, "-b", "-p", conda_path])
    logger.info('done')

    logger.info("installing rdkit")
    subprocess.check_call([
        os.path.join(conda_path, "bin", "conda"),
        "install",
        "--yes",
        "-c", "rdkit",
        "python=={}".format(python_version),
        "rdkit" if rdkit_version is None else "rdkit=={}".format(rdkit_version)])
    logger.info("done")

    import rdkit
    logger.info("rdkit-{} installation finished!".format(rdkit.__version__))


if __name__ == "__main__":
  install()

add /root/miniconda/lib/python3.6/site-packages to PYTHONPATH
python version: 3.6.9
fetching installer from https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
done
installing miniconda to /root/miniconda
done
installing rdkit
done
rdkit-2020.09.1 installation finished!


In [3]:
import numpy
import random
import pandas
import torch

from torch_geometric.data import Data

from sklearn import preprocessing
from sklearn.metrics import r2_score

from tqdm import tqdm
from mendeleev import get_table


from rdkit import Chem
from rdkit.Chem.Descriptors import ExactMolWt

In [6]:
# Experiment settings
gnn = 'gcn'
dataset_name = "/content/drive/MyDrive/0_한동생활/2020-2.5 겨울방학/2_GNN/0_Tutorial/esol.xlsx"
batch_size = 32
init_lr = 1e-4
l2_coeff = 1e-7
n_epochs = 300

In [5]:
from mendeleev import get_table
import pandas
import numpy

ptable = get_table('elements')
ptable.info()
# ptable['Column']
feature_list = ptable.columns.values
feature_list

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118 entries, 0 to 117
Data columns (total 70 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   annotation                     118 non-null    object 
 1   atomic_number                  118 non-null    int64  
 2   atomic_radius                  90 non-null     float64
 3   atomic_volume                  91 non-null     float64
 4   block                          118 non-null    object 
 5   boiling_point                  96 non-null     float64
 6   density                        95 non-null     float64
 7   description                    109 non-null    object 
 8   dipole_polarizability          117 non-null    float64
 9   electron_affinity              77 non-null     float64
 10  electronic_configuration       118 non-null    object 
 11  evaporation_heat               88 non-null     float64
 12  fusion_heat                    75 non-null     flo

array(['annotation', 'atomic_number', 'atomic_radius', 'atomic_volume',
       'block', 'boiling_point', 'density', 'description',
       'dipole_polarizability', 'electron_affinity',
       'electronic_configuration', 'evaporation_heat', 'fusion_heat',
       'group_id', 'lattice_constant', 'lattice_structure',
       'melting_point', 'name', 'period', 'series_id', 'specific_heat',
       'symbol', 'thermal_conductivity', 'vdw_radius',
       'covalent_radius_cordero', 'covalent_radius_pyykko', 'en_pauling',
       'en_allen', 'jmol_color', 'cpk_color', 'proton_affinity',
       'gas_basicity', 'heat_of_formation', 'c6', 'covalent_radius_bragg',
       'vdw_radius_bondi', 'vdw_radius_truhlar', 'vdw_radius_rt',
       'vdw_radius_batsanov', 'vdw_radius_dreiding', 'vdw_radius_uff',
       'vdw_radius_mm3', 'abundance_crust', 'abundance_sea',
       'molcas_gv_color', 'en_ghosh', 'vdw_radius_alvarez', 'c6_gb',
       'atomic_weight', 'atomic_weight_uncertainty', 'is_monoisotopic',
      

In [6]:
object_removed = ptable.select_dtypes(exclude=['object'])
removed_column_list =object_removed.columns.values 
print(removed_column_list)
print("Shape =",object_removed.shape)
object_removed.to_excel("object_removed_element_feature.xlsx")

['atomic_number' 'atomic_radius' 'atomic_volume' 'boiling_point' 'density'
 'dipole_polarizability' 'electron_affinity' 'evaporation_heat'
 'fusion_heat' 'group_id' 'lattice_constant' 'melting_point' 'period'
 'series_id' 'specific_heat' 'thermal_conductivity' 'vdw_radius'
 'covalent_radius_cordero' 'covalent_radius_pyykko' 'en_pauling'
 'en_allen' 'proton_affinity' 'gas_basicity' 'heat_of_formation' 'c6'
 'covalent_radius_bragg' 'vdw_radius_bondi' 'vdw_radius_truhlar'
 'vdw_radius_rt' 'vdw_radius_batsanov' 'vdw_radius_dreiding'
 'vdw_radius_uff' 'vdw_radius_mm3' 'abundance_crust' 'abundance_sea'
 'en_ghosh' 'vdw_radius_alvarez' 'c6_gb' 'atomic_weight'
 'atomic_weight_uncertainty' 'is_radioactive' 'atomic_radius_rahm'
 'metallic_radius' 'metallic_radius_c12' 'covalent_radius_pyykko_double'
 'covalent_radius_pyykko_triple' 'discovery_year' 'mendeleev_number'
 'dipole_polarizability_unc' 'pettifor_number' 'glawe_number']
Shape = (118, 51)


In [ ]:
elem_feats = numpy.nan_to_num(numpy.array(object_removed))
print(elem_feats.shape)
scaled_feats = preprocessing.scale(elem_feats)
# pandas.DataFrame(scaled_feats)

(118, 51)


In [ ]:
# # 좀 많이 오래 걸리는 코드 
import seaborn as sns; sns.set(style="ticks", color_codes=True)
# g = sns.pairplot(object_removed)

In [ ]:
elem_feat_names = ['atomic_weight', 'atomic_radius', 'atomic_volume', 'dipole_polarizability',
                   'fusion_heat', 'thermal_conductivity', 'vdw_radius', 'en_pauling',
                   'boiling_point','electron_affinity','en_allen','en_ghosh']
n_atom_feats = len(elem_feat_names)
tb_atom_feats = get_table('elements')
feat12 = tb_atom_feats[elem_feat_names]
feat12 = pandas.DataFrame(preprocessing.scale(feat12), columns = elem_feat_names)
g = sns.pairplot(feat12)

In [6]:
g = sns.pairplot(feat8)

NameError: ignored

In [10]:
elem_feat_names = ['atomic_weight', 'atomic_radius', 'atomic_volume', 'dipole_polarizability',
                   'fusion_heat', 'thermal_conductivity', 'vdw_radius', 'en_pauling']
n_atom_feats = len(elem_feat_names)
tb_atom_feats = get_table('elements')
feat8 = tb_atom_feats[elem_feat_names]

# feat8 = pandas.DataFrame(feat8)
g = sns.pairplot(feat8)

NameError: ignored

In [11]:
# RdKit Code Test 

n2 = Chem.MolFromSmiles("N#N")
print("n2 Bonds num", n2.GetNumBonds()) # 1 
for bond in n2.GetBonds():
  print(bond.GetBondType()) # TRIPLE

o2 = Chem.MolFromSmiles('O=O')
print("o2 Bonds num",o2.GetNumBonds()) # 1 
for bond in o2.GetBonds():
  print(bond.GetBondType()) # DOUBlE

cl2 = Chem.MolFromSmiles("ClCl")
print("cl2 Bonds num",h2.GetNumBonds()) # 1 
for bond in h2.GetBonds():
  print(bond.GetBondType()) # SINGLE

CHBrClF = Chem.MolFromSmiles("BrC(F)Cl")
for stereo in CHBrClF.GetStereoGroups():
  print(stereo)

n2 Bonds num 1
TRIPLE
o2 Bonds num 1
DOUBLE


NameError: ignored

In [ ]:
# RdKit Code Test 
from rdkit.Chem import AllChem
num_max_atoms = 30
h2o = Chem.MolFromSmiles("C(=O)=O")
h2o.ComputeGasteigerCharges()
n = h2o.GetNumAtoms()
print(n)
partial_charge_vec = numpy.zeros((num_max_atoms, 1), dtype=numpy.float32)

for i in range(n):
    a = mol.GetAtomWithIdx(i)
    print(a)
    print(a.GetPropsAsDict())
    print(a.GetPropNames())
#     partial_charge_vec[i, 0] = a.GetProp("_GasteigerCharge")
# return partial_charge_vec

# n2 = Chem.MolFromSmiles("N#N")
# print("n2 ComputeGasteigerCharges", n2.)# 1 


# o2 = Chem.MolFromSmiles('O=O')
# print("o2 ComputeGasteigerCharges",o2.ComputeGasteigerCharges()) # 1 


# cl2 = Chem.MolFromSmiles("ClCl")
# print("cl2 ComputeGasteigerCharges",o2.ComputeGasteigerCharges()) # 1 

# CHBrClF = Chem.MolFromSmiles("BrC(F)Cl")
# print("CHBrClF ComputeGasteigerCharges",o2.ComputeGasteigerCharges()) # 1 

3
{'__computedProps': <rdkit.rdBase._vectNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE object at 0x7fa4a19f4990>, '_CIPRank': 0}
{'__computedProps': <rdkit.rdBase._vectNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE object at 0x7fa4a19f4990>, '_CIPRank': 0}


RuntimeError: ignored

In [8]:
id_target = numpy.array(pandas.read_excel(dataset_name))
smiles = id_target[:, 0]
# mol = Chem.MolFromSmiles(smiles)
atom_list = []
LOHC_vocab = []
max_str_len = 0
max_num = 0 
max_degree = 0
max_long_mol = None
for smile in tqdm(smiles):
  mol = Chem.MolFromSmiles(smile)

  for char in smile:
    if char in LOHC_vocab :
      continue
    LOHC_vocab.append(char)
  
  for atom in mol.GetAtoms(): # mol Object -> Atom obj.   
    if atom.GetSymbol() in atom_list: # Symbol =. Atom Name 
      continue
    atom_list.append(atom.GetSymbol())

  
  if mol.GetNumAtoms() > max_num :
    max_num = mol.GetNumAtoms()

  if atom.GetDegree() > max_degree :
    max_degree = atom.GetDegree() 

  if mol.GetNumAtoms() == 55:
    max_long_mol = smile

  if len(smile) > max_str_len:
    max_str_len = len(smile)


print("\n")
print("가지고 있는 데이터의 원자 종류 ", atom_list)
print("데이터의 한 분자내 최대 원자수 ", max_num)
print("제일 긴 애 ",max_long_mol)
print("결합 제일 많이 되어 있는 애 결합 수", max_degree)
print("LOHC_ vocab", LOHC_vocab)
print("max_len str :",max_str_len)

100%|██████████| 1128/1128 [00:00<00:00, 2353.02it/s]



가지고 있는 데이터의 원자 종류  ['O', 'C', 'N', 'S', 'Cl', 'P', 'F', 'I', 'Br']
데이터의 한 분자내 최대 원자수  55
제일 긴 애  CC1OC(CC(O)C1O)OC2C(O)CC(OC2C)OC8C(O)CC(OC7CCC3(C)C(CCC4C3CC(O)C5(C)C(CCC45O)C6=CC(=O)OC6)C7)OC8C
결합 제일 많이 되어 있는 애 결합 수 3
LOHC_ vocab ['O', 'C', '3', '(', '2', '#', 'N', ')', 'c', '1', 'o', '=', '4', '5', 's', 'n', 'l', 'P', 'S', '[', 'H', ']', 'F', 'I', 'B', 'r', '/', '\\', '6', '7', '8']
max_len str : 97


In [8]:
elem_feat_names = ['atomic_weight', 'atomic_radius', 'atomic_volume', 'dipole_polarizability',
                   'fusion_heat', 'thermal_conductivity', 'vdw_radius', 'en_pauling']
n_atom_feats = len(elem_feat_names)

def get_elem_feats():
    tb_atom_feats = get_table('elements')
    elem_feats = numpy.nan_to_num(numpy.array(tb_atom_feats[elem_feat_names]))

    return preprocessing.scale(elem_feats)

def load_dataset(path_user_dataset):
    elem_feats = get_elem_feats()
    list_mols = list()
    id_target = numpy.array(pandas.read_excel(path_user_dataset))

    for i in tqdm(range(0, id_target.shape[0])):
        mol = smiles_to_mol_graph(elem_feats, id_target[i, 0], idx=i, target=id_target[i, 1])

        if mol is not None:
            list_mols.append((id_target[i, 0], mol))

    return list_mols

def smiles_to_mol_graph(elem_feats, smiles, idx, target):
    try:
        mol = Chem.MolFromSmiles(smiles)
        adj_mat = Chem.GetAdjacencyMatrix(mol)
        
        # adding Id
        adj_mat = adj_mat + numpy.eye(len(adj_mat))
        print(adj_mat)
        atom_feats = list()
        bonds = list()

        for atom in mol.GetAtoms():
            atom_feats.append(elem_feats[atom.GetAtomicNum() - 1, :])

        for i in range(0, mol.GetNumAtoms()):
            for j in range(0, mol.GetNumAtoms()):
                if adj_mat[i, j] == 1:
                    bonds.append([i, j])

        if len(bonds) == 0:
            return None

        atom_feats = torch.tensor(atom_feats, dtype=torch.float)
        bonds = torch.tensor(bonds, dtype=torch.long).t().contiguous()
        y = torch.tensor(target, dtype=torch.float).view(1, 1)
        mol_wt = torch.tensor(ExactMolWt(mol), dtype=torch.float).view(1, 1)
        n_rings = torch.tensor(mol.GetRingInfo().NumRings(), dtype=torch.float).view(1, 1)

        return Data(x=atom_feats, y=y, edge_index=bonds, idx=idx, mol_wt=mol_wt, n_rings=n_rings)
    except:
        return None

In [9]:
# Load dataset
print('Load molecular structures...')
data = load_dataset(dataset_name)
random.shuffle(data)
smiles = [x[0] for x in data]
mols = [x[1] for x in data]

  3%|▎         | 34/1128 [00:00<00:03, 336.51it/s]

Load molecular structures...
[[1. 1. 0. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 1. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 1. ... 0. 1. 1.]
 [0. 0. 0. ... 0. 1. 1.]]
[[1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1.]]
[[1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 0. 0. 0. 0.

  9%|▉         | 101/1128 [00:00<00:03, 333.07it/s]

[[1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 0.]
 [0. 0. 0.

 15%|█▌        | 170/1128 [00:00<00:02, 337.46it/s]

...

[[1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0

 96%|█████████▋| 1086/1128 [00:03<00:00, 278.47it/s]

[[1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0.

100%|██████████| 1128/1128 [00:04<00:00, 273.31it/s]

[[1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0.

In [ ]:
# Generate training and test datasets
n_train = int(0.8 * len(data))
train_data = mols[:n_train]
test_data = mols[n_train:]
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size)
test_smiles = smiles[n_train:]
test_targets = numpy.array([x.y.item() for x in test_data]).reshape(-1, 1)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import GATConv
from torch_geometric.nn import GINConv
from torch_geometric.nn import global_mean_pool
from torch_geometric.nn import BatchNorm

class EGCN(nn.Module):
    # extended graph convolutional network
    def __init__(self, num_node_feats, dim_out):
        super(EGCN, self).__init__()
        self.gc1 = GCNConv(num_node_feats, 256)
        self.gc2 = GCNConv(256, 256)
        self.gc3 = GCNConv(256, 256)
        self.fc1 = nn.Linear(256 + 2, 196)
        self.fc2 = nn.Linear(196, dim_out)
        self.norm = BatchNorm(256)


    def forward(self, g):
        id = g 
        h1 = F.relu(self.gc1(g.x, g.edge_index))
        h2 = F.relu(self.gc2(h1, g.edge_index))
        h3 = F.relu(self.gc3(h2, g.edge_index))
        h = h1 + h2 + h3 
        hg = global_mean_pool(h, g.batch)
        hg = torch.cat([hg, g.mol_wt, g.n_rings], dim=1)
        hg = F.relu(self.fc1(hg))
        out = self.fc2(hg)

        return out

In [ ]:
model = EGCN(n_atom_feats, 1)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=init_lr, weight_decay=l2_coeff)
criterion = torch.nn.L1Loss()

In [ ]:
def train(model, optimizer, data_loader, criterion):
    model.train()
    train_loss = 0

    for i, (batch) in enumerate(data_loader):
        pred = model(batch)
        loss = criterion(batch.y, pred)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.detach().item()

    return train_loss / len(data_loader)

In [ ]:
def test(model, data_loader):
    model.eval()
    list_preds = list()

    with torch.no_grad():
        for batch in data_loader:
            preds = model(batch)
            list_preds.append(preds)

    return torch.cat(list_preds, dim=0).cpu().numpy()

In [ ]:
# Train graph neural network (GNN)
print('Train the GNN-based predictor...')
for i in range(0, 500):
    train_loss = train(model, optimizer, train_loader, criterion)
    print('Epoch [{}/{}]\tTrain loss: {:.4f}'.format(i + 1, n_epochs, train_loss))

In [ ]:
# Test the trained GNN
preds = test(model, test_loader)
test_mae = numpy.mean(numpy.abs(test_targets - preds))
r2 = r2_score(test_targets, preds)
print('Test MAE: {:.4f}\tTest R2 score: {:.4f}'.format(test_mae, r2))

In [ ]:
# Test the trained GNN
preds = test(model, test_loader)
test_mae = numpy.mean(numpy.abs(test_targets - preds))
r2 = r2_score(test_targets, preds)
print('Test MAE: {:.4f}\tTest R2 score: {:.4f}'.format(test_mae, r2))